# Modules to be imported

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import overpass
import psycopg2
import statistics
import tensorflow as tf
from tensorflow import keras

# Webscraping from www.immoscout24.ch

In [ ]:
# Create an empty array to store the scraped data
properties = []
for page in range(1, 3):
# Create a new instance of Chrome driver
    driver = webdriver.Chrome()

# Navigate to the search result page for Zurich city on ImmoScout24.ch

    print(page)
        # List of URLs to scrape
    urls = [
    'https://www.immoscout24.ch/en/real-estate/rent/city-zuerich?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-geneve?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-basel?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-lausanne?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-bern?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-winterthur?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-luzern?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-st-gallen?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-lugano?pn='+str(page),
    'https://www.immoscout24.ch/en/real-estate/rent/city-biel-bienne?pn='+str(page)
]

    # loop thorugh each URL 

    for url in urls:
               
        driver.get(url)

        # Wait for the page to load completely
        driver.implicitly_wait(10)

        # Use BeautifulSoup to parse the HTML content of the page
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Find all the property listings on the page
        listings = soup.find_all("a", {"class": "Wrapper__A-kVOWTT"})
        

    # Loop through the listings and extract the required information
    for listing in listings:
        #print(listing)
        # Get the price, size, and address of the property
        price = listing.find("h3", {"class": re.compile("^Box-cYFBPY hKJGPR Heading")}).text.strip()
        #size = listing.find("h3", {"class": re.compile("^Box-cYFBPY hKJGPR Heading")}).text.strip()
        address = listing.find("span", {"class": "AddressLine__TextStyled-eaUAMD"}).text.strip()
        
        # Add the property details to the properties array as a tuple
        properties.append((price, address))

    # Close the Chrome driver
    driver.close()

    # Print the scraped data array
    print("Properties:", properties)

1
2
Properties: [('2.5 rooms, 55 m², CHF 2600.—', 'Schaffhauserstrasse 453, 8052 Zürich, ZH'), ('2.5 rooms, 54 m², CHF 2500.—', 'Schaffhauserstrasse 453, 8052 Zürich, ZH'), ('3.5 rooms, 69 m², CHF 3750.—', 'Nelkenstrasse 20, 8006 Zürich, ZH'), ('1.5 rooms, 23 m², CHF 1460.—', 'Viktoriastrasse 29, 8057 Zürich, ZH'), ('2 rooms, 48 m², CHF 2720.—', 'Zelgstrasse 41, 8003 Zürich, ZH'), ('3.5 rooms, 135 m², CHF 7150.—', 'Kraftstrasse 37, 8044 Zürich, ZH'), ('2.5 rooms, 76 m², CHF 3970.—', 'Schöntalstrasse 20, 8004 Zürich, ZH'), ('4.5 rooms, 156 m², CHF 5450.—', 'Alfred Escher-Strasse 25, 8002 Zürich, ZH'), ('4.5 rooms, 125 m², CHF 3600.—', 'Eisfeldstrasse 30, 8052 Zürich, ZH'), ('4.5 rooms, 110 m², CHF 5960.—', 'Turnerstrasse 18, 8006 Zürich, ZH'), ('1 room, CHF 1300.—', 'Baslerstrasse 110, 8048 Zürich, ZH'), ('4 rooms, 101 m², CHF 3625.—', 'Baslerstrasse 77, 8048 Zürich, ZH'), ('4.5 rooms, 84 m², CHF 2940.—', 'Rautistrasse 332, 8048 Zürich, ZH'), ('4.5 rooms, 117 m², CHF 3270.—', 'Badenerst

# WEB API using Overpass Turbo

In [ ]:
# defining the overpass query
query = '''
[out:json];
{{geocodeArea:Zurich}} -> .searchArea0;
{{geocodeArea:Geneva}} -> .searchArea1;
{{geocodeArea:Basel}} -> .searchArea2;
{{geocodeArea:Lausanne}} -> .searchArea3;
{{geocodeArea:Bern}} -> .searchArea4;
{{geocodeArea:Winterthur}} -> .searchArea5;
{{geocodeArea:Luzern}} -> .searchArea6;
{{geocodeArea:St. Gallen}} -> .searchArea7;
{{geocodeArea:Lugano}} -> .searchArea8;
{{geocodeArea:Biel}} -> .searchArea9;
( area.searchArea0; area.searchArea1; area.searchArea2; area.searchArea3; area.searchArea4; area.searchArea5; area.searchArea6; area.searchArea7; area.searchArea8; area.searchArea9;) -> .searchArea;
(node["shop"="supermarket"](area.searchArea);way["shop"="supermarket"](area.searchArea););
out center;
'''

# creating Overpass API object
api = overpass.API()

# performing the query and retrieveing the results
response = api.get(query)

# print the name and zip code of each supermarket
for element in response["elements"]:
    if element["tags"].get("name") and element["tags"].get("addr:postcode"):
        name = element["tags"]["name"]
        zip_code = element["tags"]["addr:postcode"]
        print(f"{name}: {zip_code}")

# Connection to AWS PostgreSQL

In [ ]:
#Connection to the database
conn = psycopg2.connect(
    host="ads-database.cbwqb3cep5ch.eu-central-1.rds.amazonaws.com",
    database="ads-database",
    user="adschief",
    password="eYvmDnMPikKSImeLrev"
)

# Creating a cursor object for executing SQL statements
cur = conn.cursor()

# Iterate over the properties array
for property in properties:
    # Extract the property details
    details = property[0].split(", ")  # Split the details by comma and space

    # Extract the individual details
    rooms = details[0]
    size = details[1]
    price = details[2]
    address = details[3]
    zipcode = details[4]
    kanton = details [5]
    
    # Process the individual data record as needed
    print("Rooms:", rooms)
    print("Size:", size)
    print("Price:", price)
    print("Address:", address)
    print("Zip-Code", zipcode)
    print("Kanton", kanton)
    print("--------------------")


    # Insert the data into the database
    cur.execute(
        "INSERT INTO your_table_name (rooms, size, price, address, zipcode, kanton) VALUES (%s, %s, %s, %s. %s, %s)",
        (rooms, size, price, address, zipcode, kanton)
    )
    
    # Commit the changes to the database
conn.commit()

# Close the cursor and connection
#cur.close()
#conn.close()


# EDA using our data from PostgreSQL

In [ ]:
#mean, median, mean, median, mode, variance, standard deviation
#histogram, boxplots, scatter plots
#correlation analysis
#hypothesis testing 

# Retrieve the data from the database
cur.execute("SELECT price FROM XXXXXXXXXXXX")

# Fetch all the rows of the query result
rows = cur.fetchall()

# Extract the values from the rows
prices = [row[0] for row in rows]

# Calculate the mean
mean = statistics.mean(prices)

# Calculate the median
median = statistics.median(prices)

# Calculate the mode
mode = statistics.mode(prices)

# Calculate the variance
variance = statistics.variance(prices)

# Calculate the standard deviation
std_dev = statistics.stdev(prices)

# Print the results
print("Mean:", mean)
print("Median:", median)
print("Mode:", mode)
print("Variance:", variance)
print("Standard Deviation:", std_dev)

# Using TensorFlow and Keras framework/library for ML